#Controlar la presencia de oligosacáridos ramificados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Biopython
!pip install pandarallel

In [ ]:
#@title Importar archivos

import pandas as pd
import os

input_folder = "/content/drive/MyDrive/TFM/T1/oligosacaridos_ramificados"
input_file = os.path.join(input_folder,"df_harm_filtro_3.csv")
df_harm_1 = pd.read_csv(input_file, sep = ',')

In [ ]:
#@title Función para comprobar si hay o no oligosacáridos ramificados

def check_oligosacaridos(row):
    entrada = row['PDB_entry_id']
    if entrada[0] in ['3']:
      subfolder = entrada[0]
      cif_file = os.path.join(input_folder, subfolder, f"{entrada.lower()}.cif")

      try:
          cif_dict = MMCIF2Dict(cif_file)
          if "_pdbx_entity_branch.type" in cif_dict:
              return True  # Contiene oligosacárido ramificado
      except Exception as e:
          print(f"Error con {entrada}: {e}")
          entrada_no_encontrada.append(entrada)

      return False

In [ ]:
#@title Dataframe con las entradas a analizar

df_lista_3 = df_harm_1[df_harm_1['PDB_entry_id'].str.startswith('3')].copy()
print(len(df_lista_3))

9068


In [ ]:
#@title Aplicar la función al dataframe y crear el dataframe de output con solo los oligosacaridos ramificados

# Inicializar pandarallel (ajusta workers si tienes más núcleos)
pandarallel.initialize(nb_workers=2, progress_bar=True)

df_lista_3['es_oligosacarido_ramificado'] = df_lista_3.parallel_apply(check_oligosacaridos, axis=1)

# Evitar error de NaN
df_lista_3['es_oligosacarido_ramificado'] = df_lista_3['es_oligosacarido_ramificado'].fillna(False)

df_oligos = df_lista_3[df_lista_3['es_oligosacarido_ramificado']]
print(f"Total encontrados: {len(df_oligos)}")

output_file = os.path.join(input_folder, "oligosacaridos_ramificados.csv")
df_oligos.to_csv(output_file, index=False)